### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="ML-for-AQM-Prediction-v2" + fablib.get_bastion_username()

node_conf = [
 {'name': "tx_main",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "tx_2",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "tx_3",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 

 {'name': "router-bottleneck", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "router-1", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "router-3", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "delay", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
    
 {'name': "rx_main",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "rx_2",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']},
 {'name': "rx_3",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}

]
net_conf = [
 {"name": "net-tx", "subnet": "10.0.0.0/24", "nodes": [{"name": "tx_main",   "addr": "10.0.0.100"}, {"name": "tx_2",   "addr": "10.0.0.101"}, {"name": "tx_3",   "addr": "10.0.0.102"}, {"name": "delay", "addr": "10.0.0.2"}]},
 {"name": "net-delay-router", "subnet": "10.0.2.0/24", "nodes": [{"name": "delay",   "addr": "10.0.2.2"}, {"name": "router-1", "addr": "10.0.2.1"}]},

 {"name": "net-router1-bottleneck", "subnet": "10.0.3.0/24", "nodes": [{"name": "router-1",   "addr": "10.0.3.2"}, {"name": "router-bottleneck", "addr": "10.0.3.1"}]},

 {"name": "net-bottleneck-router3", "subnet": "10.0.4.0/24", "nodes": [{"name": "router-bottleneck",   "addr": "10.0.4.2"}, {"name": "router-3", "addr": "10.0.4.1"}]},

 {"name": "net-rx", "subnet": "10.0.5.0/24", "nodes": [{"name": "router-3",   "addr": "10.0.5.1"}, {"name": "rx_main", "addr": "10.0.5.100"}, {"name": "rx_2", "addr": "10.0.5.101"}, {"name": "rx_3", "addr": "10.0.5.102"}]}

]
route_conf = [
 {"addr": "10.0.5.0/24", "gw": "10.0.0.2", "nodes": ["tx_main", "tx_2", "tx_3"]}, 
 {"addr": "10.0.5.0/24", "gw": "10.0.2.1", "nodes": ["delay"]},
 {"addr": "10.0.5.0/24", "gw": "10.0.3.1", "nodes": ["router-1"]},
 {"addr": "10.0.5.0/24", "gw": "10.0.4.1", "nodes": ["router-bottleneck"]},
 

 {"addr": "10.0.0.0/24", "gw": "10.0.5.1", "nodes": ["rx_main", "rx_2", "rx_3"]},
 {"addr": "10.0.0.0/24", "gw": "10.0.4.2", "nodes": ["router-3"]},
 {"addr": "10.0.0.0/24", "gw": "10.0.3.2", "nodes": ["router-bottleneck"]},
 {"addr": "10.0.0.0/24", "gw": "10.0.2.2", "nodes": ["router-1"]},

]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

In [ ]:
for node in slice.get_nodes():
    # check kernel version
    node.execute("hostname; uname -a")

In [ ]:
slice.get_node(name="router").execute("ip -V")

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Execute Experiment

In [ ]:
# nodes and instances

tx_main_node = slice.get_node(name="tx_main")
tx_2_node = slice.get_node(name="tx_2")
tx_3_node = slice.get_node(name="tx_3")

rx_main_node = slice.get_node(name="rx_main")
rx_2_node = slice.get_node(name="rx_2")
rx_3_node = slice.get_node(name="rx_3")

delay_node = slice.get_node(name="delay")
router_node = slice.get_node(name="router-bottleneck")
router1_node = slice.get_node(name="router-1")
router3_node = slice.get_node(name="router-3")

# interfaces
tx_main_egress_iface  = tx_main_node.get_interface(network_name = "net-tx")
tx_2_egress_iface  = tx_2_node.get_interface(network_name = "net-tx")
tx_3_egress_iface  = tx_3_node.get_interface(network_name = "net-tx")


delay_ingress_tx_iface  = delay_node.get_interface(network_name = "net-tx")
delay_egress_iface  = delay_node.get_interface(network_name = "net-delay-router")
delay_ingress_tx_name = delay_ingress_tx_iface.get_device_name()
delay_egress_name = delay_egress_iface.get_device_name()

router1_ingress_iface  = router1_node.get_interface(network_name = "net-delay-router")
router1_egress_iface  = router1_node.get_interface(network_name = "net-router1-bottleneck")
router1_egress_name  = router1_egress_iface.get_device_name()

router_ingress_iface  = router_node.get_interface(network_name = "net-router1-bottleneck")
router_egress_iface  = router_node.get_interface(network_name = "net-bottleneck-router3")
router_egress_name  = router_egress_iface.get_device_name()

router3_ingress_iface  = router3_node.get_interface(network_name = "net-bottleneck-router3")
router3_egress_iface  = router3_node.get_interface(network_name = "net-rx")
router3_egress_name  = router3_egress_iface.get_device_name()

rx_main_ingress_iface  = rx_main_node.get_interface(network_name = "net-rx")
rx_2_ingress_iface  = rx_2_node.get_interface(network_name = "net-rx")
rx_3_ingress_iface  = rx_3_node.get_interface(network_name = "net-rx")

In [ ]:
# generate full factorial experiment
import itertools

exp_factors = {
    'n_bdp': [1, 3],  # Novel bandwidth-delay product (smaller and larger than training)
    'btl_capacity': [40, 150],  # Intermediate and high capacities not seen in training
    'base_rtt': [15, 60],  # Include smaller and larger RTTs
    'aqm': ['FIFO','pie_drop'],
    'target': [5, 15],  # Add different AQM targets for queue delay
    'cc_tx_main': ["cubic"],  # Keep the same for consistency
    'cc_tx_2': ["cubic"],  # you can change it to BBR also
    'cc_tx_3': ["reno"],  
    'N_tx_main': [1],  # Vary number of flows for main sender
    'N_tx_2': [2, 10, 20],  # Include additional flow counts for more diversity
    'N_tx_3': [2, 10, 20],  # Similar flow counts for tx_3
    'trial': [1],  # Single trial for simplicity
}

factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))

exp_lists = []

seen_combinations = set()

# Removing target from FIFO bottleneck because it does not have target
# Removing the cases where target is more than or equal to the buffer size in time, these cases are not meaningful in practice

for factor_l in factor_lists:
    temp_dict = dict(zip(factor_names, factor_l))
    if temp_dict['n_bdp'] * temp_dict['base_rtt'] >= temp_dict['target']:
        if temp_dict['aqm'] == 'FIFO':
            del temp_dict['target']
        # Convert dict to a frozenset for set operations
        fs = frozenset(temp_dict.items())
    
        if fs not in seen_combinations:
            seen_combinations.add(fs)
            exp_lists.append(temp_dict)

data_dir_tx_main = slice_name + 'singlebottleneck'+"-tx_main"
data_dir_tx_2 = slice_name + 'singlebottleneck'+"-tx_2"
data_dir_tx_3 = slice_name + 'singlebottleneck'+"-tx_3"


print("Number of experiments:",len(exp_lists))

In [ ]:
#tx_main_node.execute("sudo modprobe tcp_" + exp['cc_tx_main'])
tx_main_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp_factors['cc_tx_main'][0])

#tx_2_node.execute("sudo modprobe tcp_" + exp['cc_tx_2'])
tx_2_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp_factors['cc_tx_2'][0])

#tx_3_node.execute("sudo modprobe tcp_" + exp['cc_tx_3'])
tx_3_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp_factors['cc_tx_3'][0])


rx_main_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp_factors['cc_tx_main'][0])
rx_2_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp_factors['cc_tx_2'][0])
rx_3_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp_factors['cc_tx_3'][0])

In [ ]:
# run experiments
import time
d = 20 #duration in seconds

em = [delay_ingress_tx_name, delay_egress_name]

for exp in exp_lists:

    # check if we already ran this experiment
    # (allow stop/resume)
    name_tx_main="%s_%s_%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (exp['cc_tx_main'],exp['cc_tx_2'],exp['cc_tx_3'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('target', 'none')), exp['N_tx_main'],exp['N_tx_2'],exp['N_tx_3'],exp['trial'])
    name_tx_2="%s_%s_%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % ("2-"+exp['cc_tx_main'],exp['cc_tx_2'],exp['cc_tx_3'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('target', 'none')), exp['N_tx_main'],exp['N_tx_2'],exp['N_tx_3'],exp['trial'])
    name_tx_3="%s_%s_%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % ("3-"+exp['cc_tx_main'],exp['cc_tx_2'],exp['cc_tx_3'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('target', 'none')), exp['N_tx_main'],exp['N_tx_2'],exp['N_tx_3'],exp['trial'])


    #name_tx_legacy="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (exp['cc_tx_legacy'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx_L4S_ecn'], exp['rx_legacy_ecn'], exp['trial'])
    
    
    
    file_out_tx_main_json = name_tx_main+"-result.json"
    stdout_tx_main_json, stderr_tx_main_json = tx_main_node.execute("ls " + file_out_tx_main_json, quiet=True) 
    
    

    if len(stdout_tx_main_json):
        print("Already have " + name_tx_main + ", skipping")

    elif len(stderr_tx_main_json):
        print("Running experiment to generate " + name_tx_main) 

        
        # delay at emulator
        for e in em:
            cmds = "sudo tc qdisc replace dev {iface} root netem delay {owd}ms limit 60000".format(iface=e, owd=exp['base_rtt']/2)
            delay_node.execute(cmds)
        
        # fixed values
        btl_limit    = int(1000*exp['n_bdp']*exp['btl_capacity']*exp['base_rtt']/8) # limit of the bottleneck, n_bdp x BDP in bytes 
        packet_number=int(btl_limit/1500)+1
        

        #aqm type selection
        cmds_prefix = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], buffer=btl_limit)
        
        cmds_specific_initial = "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: ".format(iface=router_egress_name)

        cmds_specific = {
        'FIFO': "bfifo limit {buffer}".format(buffer=btl_limit),
        'single_queue_FQ': "fq limit {packet_limit} flow_limit {packet_limit} orphan_mask 0 ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'Codel': "codel limit {packet_limit} target {target}ms interval 100ms ecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'Codel_drop': "codel limit {packet_limit} target {target}ms interval 100ms noecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'FQ': "fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'FQ_Codel': "fq_codel limit {packet_limit} target {target}ms interval 100ms ecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'FQ_Codel_L4S': "fq_codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold 1ms ce_threshold_selector 0x01/0x01".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'pie_drop': "pie limit {packet_limit} target {target}ms noecn".format(packet_limit=packet_number, target=exp.get('target', 0)),
        'DualPI2': "dualpi2 target {threshold}ms".format(threshold=exp.get('ecn_threshold', 0))
        }

        cmds_aqm = {key: cmds_specific_initial + cmd for key, cmd in cmds_specific.items()}
        
        router_node.execute(cmds_prefix)
        router_node.execute(cmds_aqm[ exp['aqm'] ])
            
        rx_main_node.execute("killall iperf3")
        rx_2_node.execute("killall iperf3")
        rx_3_node.execute("killall iperf3")

        ss_tx_main_script="rm -f {flow}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipn dst 10.0.5.100 | ts '%.s' | tee -a {flow}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"
        

        #print("Starting experiment with {1} bdp {2} capacity {3} rtt {4} {5} thrshold {6} ecn_fallback {7} rx_L4S {8} rx_legacy for {duration} seconds".format(duration=d, 1=exp['n_bdp'], 2=exp['btl_capacity'], 3=exp['base_rtt'], 4=exp['aqm'], 5=exp['ecn_threshold'], 6= exp['ecn_fallback'], 7=exp['rx_L4S_ecn'], 8=exp['rx_legacy_ecn']))
        
        rx_main_node.execute("iperf3 -s -1 -p 4000 -D")
        rx_2_node.execute("iperf3 -s -1 -p 5000 -D")
        rx_3_node.execute("iperf3 -s -1 -p 6000 -D")

        tx_main_node.execute_thread("sleep 1; iperf3 -c 10.0.5.100 -t {duration} -P {flows} -C {cc} -p 4000 -J > {flow}-result.json".format(flow =name_tx_main, duration=d, flows=exp['N_tx_main'], cc=exp['cc_tx_main']))
        tx_main_node.execute_thread(ss_tx_main_script.format(flow=name_tx_main, duration=d))

        tx_3_node.execute_thread("sleep 1; iperf3 -c 10.0.5.102 -t {duration} -P {flows} -C {cc} -p 6000 -J > {flow}-result.json".format(flow =name_tx_3, duration=d, flows=exp['N_tx_3'], cc=exp['cc_tx_3']))
        stdout, stderr = tx_2_node.execute("sleep 1; iperf3 -c 10.0.5.101 -t {duration} -P {flows} -C {cc} -p 5000 -J > {flow}-result.json".format(flow =name_tx_2, duration=d, flows=exp['N_tx_2'], cc=exp['cc_tx_2']))
        time.sleep(3)  # time.sleep(1)
        
print("finished")

### Analysis of the results

In [ ]:
def generate_script(name): 
    return """
        rm -f {name}-ss.csv
        cat {name}-ss.txt | sed -e ":a; /<->$/ {{ N; s/<->\\n//; ba; }}"  | grep "iperf3" | grep -v "SYN-SENT" > {name}-ss-processed.txt 
        cat {name}-ss-processed.txt | awk '{{print $1}}' > ts-{name}.txt 
        cat {name}-ss-processed.txt | grep -oP 'cwnd:\\d+' | awk -F ':' '{{print $2}}' > cwnd-{name}.txt 
        cat {name}-ss-processed.txt | grep -oP '\\brtt:.*?(\\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' '  | cut -d '/' -f 1 > srtt-{name}.txt 
        cat {name}-ss-processed.txt | grep -oP 'fd=.*?(\\s|$)' | awk -F '[=,]' '{{print $2}}' | tr -d ')' | tr -d ' '   > fd-{name}.txt
        paste ts-{name}.txt fd-{name}.txt cwnd-{name}.txt srtt-{name}.txt -d ',' > {name}-ss.csv
    """.format(name=name)

In [ ]:
for exp in exp_lists:
    
    name_tx_main="%s_%s_%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (exp['cc_tx_main'],exp['cc_tx_2'],exp['cc_tx_3'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('target', 'none')), exp['N_tx_main'],exp['N_tx_2'],exp['N_tx_3'],exp['trial'])

    file_out_tx_main_csv = name_tx_main+"-ss.csv"
    stdout_tx_main_csv, stderr_tx_main_csv = tx_main_node.execute("ls " + file_out_tx_main_csv, quiet=True) 

    if len(stdout_tx_main_csv):
        print("Already have " + name_tx_main + ", skipping")

    elif len(stderr_tx_main_csv):
        print("Running experiment to generate " + name_tx_main) 

    
        script_tx_main = generate_script(name_tx_main)
        tx_main_node.execute(script_tx_main)

In [ ]:
tx_main_node.execute('mkdir '+data_dir_tx_main)
tx_main_node.execute('mv *.json '+ data_dir_tx_main)
tx_main_node.execute('mv *.csv '+ data_dir_tx_main)
tx_main_node.execute('mv *.txt '+ data_dir_tx_main)

In [ ]:
tx_main_node.upload_file("/home/fabric/work/ML for AQM detection/analysis_with_ss.py", f"/home/ubuntu/{data_dir_tx_main}/analysis_with_ss.py")

In [ ]:
cmds_py_install = '''
            sudo apt -y install python3-pip
            pip install numpy
            pip install matplotlib
            pip install pandas
            '''

tx_main_node.execute(cmds_py_install)

In [ ]:
tx_main_node.execute(f"chmod +x /home/ubuntu/{data_dir_tx_main}/analysis_with_ss.py")
tx_main_node.execute(f"python3 /home/ubuntu/{data_dir_tx_main}/analysis_with_ss.py")

In [ ]:
tx_main_node.download_file("/home/fabric/work/ML for AQM detection/tput_tx_test.json",f"/home/ubuntu/{data_dir_tx_main}/throughput_data.json")
tx_main_node.download_file("/home/fabric/work/ML for AQM detection/srtt_tx_test.json",f"/home/ubuntu/{data_dir_tx_main}/srtt_data.json")

In [ ]:
tx_main_node.download_file("/home/fabric/work/ML for AQM detection/cwnd_test.csv",f"/home/ubuntu/{data_dir_tx_main}/consolidated_cwnd_data.csv")
tx_main_node.download_file("/home/fabric/work/ML for AQM detection/time_test.csv",f"/home/ubuntu/{data_dir_tx_main}/consolidated_time_data.csv")
tx_main_node.download_file("/home/fabric/work/ML for AQM detection/srtt_test.csv",f"/home/ubuntu/{data_dir_tx_main}/srtt_all.csv")

In [ ]:
import json
import os

# Initialize empty variables
throughput_data = {}
srtt_data = {}

# Directory containing JSON files
#data_directory = '/home/fabric/work/'

data_directory = '/home/fabric/work/ML for AQM detection/'
# List of JSON files in the directory
json_files = [f for f in os.listdir(data_directory) if f.endswith('.json')]

# Load data from each JSON file and update the variables
for file_name in json_files:
    file_path = os.path.join(data_directory, file_name)
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Check if the file contains throughput data or srtt data based on its name
    if 'tput' in file_name:
        throughput_data.update(data)
    elif 'srtt' in file_name:
        srtt_data.update(data)

In [ ]:
srtt_data